In [ ]:
%%configure -f
{"conf":{"spark.driver.extraClassPath":"/usr/lib/hadoop-lzo/lib/*:/usr/lib/hadoop/hadoop-aws.jar:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/usr/share/aws/emr/security/conf:/usr/share/aws/emr/security/lib/*:/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar:/docker/usr/lib/hadoop-lzo/lib/*:/docker/usr/lib/hadoop/hadoop-aws.jar:/docker/usr/share/aws/aws-java-sdk/*:/docker/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/docker/usr/share/aws/emr/security/conf:/docker/usr/share/aws/emr/security/lib/*:/docker/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/docker/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/docker/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/docker/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar,file:///opt/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar:file:///opt/benchmark-tools/oap/oap_jars/spark-arrow-datasource-standard-1.2.0-jar-with-dependencies.jar:file:///opt/benchmark-tools/oap/oap_jars/spark-columnar-core-1.2.0-jar-with-dependencies.jar", 
        "spark.executor.extraClassPath":"/usr/lib/hadoop-lzo/lib/*:/usr/lib/hadoop/hadoop-aws.jar:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/usr/share/aws/emr/security/conf:/usr/share/aws/emr/security/lib/*:/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar:/docker/usr/lib/hadoop-lzo/lib/*:/docker/usr/lib/hadoop/hadoop-aws.jar:/docker/usr/share/aws/aws-java-sdk/*:/docker/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/docker/usr/share/aws/emr/security/conf:/docker/usr/share/aws/emr/security/lib/*:/docker/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/docker/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/docker/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/docker/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar,file:///opt/benchmark-tools/spark-sql-perf/target/scala-2.12/spark-sql-perf_2.12-0.5.1-SNAPSHOT.jar:file:///opt/benchmark-tools/oap/oap_jars/spark-arrow-datasource-standard-1.2.0-jar-with-dependencies.jar:file:///opt/benchmark-tools/oap/oap_jars/spark-columnar-core-1.2.0-jar-with-dependencies.jar",
        "spark.driver.extraLibraryPath":"/usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native:/docker/usr/lib/hadoop/lib/native:/docker/usr/lib/hadoop-lzo/lib/native:/opt/benchmark-tools/oap/lib",
        "spark.executor.extraLibraryPath":"/usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native:/docker/usr/lib/hadoop/lib/native:/docker/usr/lib/hadoop-lzo/lib/native:/opt/benchmark-tools/oap/lib",
        "spark.executor.defaultJavaOptions":"-verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:OnOutOfMemoryError='kill -9 %p' -XX:MaxDirectMemorySize=40G",
        "spark.executorEnv.CC":"/opt/benchmark-tools/oap/bin/gcc",
        "spark.executorEnv.CXX":"/opt/benchmark-tools/oap/bin/g++",
        "spark.executorEnv.CPLUS_INCLUDE_PATH":"/opt/benchmark-tools/oap/include",
        "spark.executorEnv.LD_LIBRARY_PATH":"/opt/benchmark-tools/oap/lib",
        "spark.sql.extensions":"com.intel.oap.ColumnarPlugin",
        "spark.shuffle.manager":"org.apache.spark.shuffle.sort.ColumnarShuffleManager",
        "spark.sql.join.preferSortMergeJoin":"false",
        "spark.sql.inMemoryColumnarStorage.batchSize":"20480",
        "spark.sql.parquet.columnarReaderBatchSize":"20480",
        "spark.sql.execution.arrow.maxRecordsPerBatch":"20480",
        "spark.executor.memoryOverhead":"512m",
        "spark.sql.autoBroadcastJoinThreshold":"10485760",
        "spark.driver.maxResultSize":"1g",
        "spark.memory.offHeap.enabled":"false",
        "spark.memory.offHeap.size":"10g",
        "spark.sql.adaptive.enabled":"false",
        "spark.sql.sources.useV1SourceList":"avro",
        "spark.driver.userClassPathFirst":"true",
        "spark.executor.userClassPathFirst":"true"}}

In [ ]:
import java.text.SimpleDateFormat;
import java.util.Date
import java.util.concurrent.Executors
import java.util.concurrent.ExecutorService
import com.databricks.spark.sql.perf.tpcds.TPCDS
import com.databricks.spark.sql.perf.Benchmark.ExperimentStatus

val stream_num = 2              // how many streams you want to start 
val scaleFactor = "1"           // data scale 1GB
val iterations = 1              // how many times to run the whole set of queries.
val format = "parquet"          // support parquer or orc
val storage = "s3"              // support hdfs or s3
var bucket_name = "aws-emr-resources-348941870272-us-east-2"   // when storage is "s3", this value will be use.
val partitionTables = true      // create partition tables
val query_filter = Seq()        // Seq() == all queries
//val query_filter = Seq("q1-v2.4", "q2-v2.4") // run subset of queries
val randomizeQueries = true     // run queries in a random order. Recommended for parallel runs.
 
if (storage == "hdfs"){
    bucket_name = "/user/livy"  // scala notebook only has the write permission of "hdfs:///user/livy" directory               
}

val current_time = new SimpleDateFormat("yyyy-MM-dd-HH:mm:ss").format(new Date)
var resultLocation = s"${storage}://${bucket_name}/results/tpcds_${format}/${scaleFactor}/${current_time}"
var databaseName = s"tpcds_${format}_scale_${scaleFactor}_db"
val use_arrow = true            // when you want to use gazella_plugin to run TPC-DS, you need to set it true.

if (use_arrow){
    val data_path = s"${storage}://${bucket_name}/datagen/tpcds_${format}/${scaleFactor}"
    resultLocation = s"${storage}://${bucket_name}/results/tpcds_arrow/${scaleFactor}/"
    databaseName = s"tpcds_arrow_scale_${scaleFactor}_db"
    val tables = Seq("call_center", "catalog_page", "catalog_returns", "catalog_sales", "customer", "customer_address", "customer_demographics", "date_dim", "household_demographics", "income_band", "inventory", "item", "promotion", "reason", "ship_mode", "store", "store_returns", "store_sales", "time_dim", "warehouse", "web_page", "web_returns", "web_sales", "web_site")
    sql(s"DROP database $databaseName CASCADE")
    if (spark.catalog.databaseExists(s"$databaseName")) {
        println(s"$databaseName has exists!")
    }else{
        spark.sql(s"create database if not exists $databaseName").show
        spark.sql(s"use $databaseName").show
        for (table <- tables) {
            if (spark.catalog.tableExists(s"$table")){
                println(s"$table has exists!")
            }else{
                spark.catalog.createTable(s"$table", s"$data_path/$table", "arrow")
            }
        }
        if (partitionTables) {
            for (table <- tables) {
                try{
                    spark.sql(s"ALTER TABLE $table RECOVER PARTITIONS").show
                }catch{
                        case e: Exception => println(e)
                }
            }
        }
    }
}

val timeout = 60 // timeout in hours
val tpcds = new TPCDS (sqlContext = spark.sqlContext)
spark.conf.set("spark.sql.broadcastTimeout", "10000") // good idea for Q14, Q88.
sql(s"use $databaseName")

def queries = {
    val filtered_queries = query_filter match {
        case Seq() => tpcds.tpcds2_4Queries
        case _ => tpcds.tpcds2_4Queries.filter(q => query_filter.contains(q.name))
    }
    if (randomizeQueries) scala.util.Random.shuffle(filtered_queries) else filtered_queries
}

class ThreadStream(experiment:ExperimentStatus) extends Thread{
    override def run(){
        println(experiment.toString)
        experiment.waitForFinish(timeout*60*60)
    }
}

val threadPool:ExecutorService=Executors.newFixedThreadPool(stream_num)
val experiments:Array[ExperimentStatus] = new Array[ExperimentStatus](stream_num)

try {
    for(i <- 0 to (stream_num - 1)){
        experiments(i) = tpcds.runExperiment(
            queries, 
            iterations = iterations,
            resultLocation = resultLocation,
            tags = Map("runtype" -> "benchmark", "database" -> databaseName, "scale_factor" -> scaleFactor)
        )
        threadPool.execute(new ThreadStream(experiments(i)))
    }
}finally{
    threadPool.shutdown()
}